In [18]:
%pip install selenium duckduckgo_search beautifulsoup4  webdriver_manager pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from duckduckgo_search import DDGS
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import json
import time
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [20]:
def extract(name, url):
    # Configure Selenium options (headless browser, if desired)
    options = Options()
    options.headless = True  # Set to False if you want to see the browser interaction

    # Specify the path to chromedriver.exe
    # driver_path = 'chromedriver.exe'  # Update with your actual path to chromedriver

    # # Initialize the WebDriver with Chrome
    # service = Service(driver_path)
    # driver = webdriver.Chrome(service=service, options=options)
    options = Options()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


    # Load the webpage
    driver.get(url)

    # Wait for the page to fully render (adjust wait time as needed)
    time.sleep(5)  # Example: Wait for 5 seconds (you can use more sophisticated waits)

    # Extract the page content after rendering
    page_source = driver.page_source

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Close the WebDriver session
    driver.quit()

    return soup

In [21]:
def clean_text(text):
    # Remove extra whitespace and newlines
    return re.sub(r'\s+', ' ', text).strip()

In [22]:
def extract_text_from_element(element):
    if element.name in ['p', 'ul', 'ol', 'table']:
        return clean_text(element.get_text())
    return ""

In [23]:
def scrape(soup):

    # Remove all script and style elements
    for script in soup(["script", "style", "img", "link", "href"]):
        script.decompose()

    parse_data = {
        "title": "",
        "meta_description": "",
        "keywords": "",
        "basic_info": {},
    }

    # Extract title
    title_tag = soup.find('title')
    if title_tag:
        parse_data["title"] = clean_text(title_tag.string)

    # Extract meta description
    meta_desc = soup.find('meta', attrs={'name': 'description'})
    if meta_desc:
        parse_data["meta_description"] = clean_text(meta_desc.get('content', ''))

    # Extract keywords
    keywords = soup.find('meta', attrs={'name': 'keywords'})
    if keywords:
        parse_data["keywords"] = clean_text(keywords.get('content', ''))

    # Extract structured data
    structured_data_tags = soup.find_all('script', type='application/ld+json')
    for tag in structured_data_tags:
        try:
            data = json.loads(tag.string)
            parse_data["structured_data"].append(data)
        except json.JSONDecodeError:
            pass  # Ignore invalid JSON

    # Extract basic information from the first section
    parse_data["basic_info"]=[]
    parse_data["Section"] = {}
    about_section = soup.find('section', class_='bg-white rounded-16 p-6')
    if about_section:
        parse_data["Section"]["name"] = clean_text(about_section.find('h2').text) if about_section.find('h2') else ""
        content_div = about_section.find('div', class_='content-section')
        if content_div:
            paragraphs = content_div.find_all('p')
            parse_data["Section"]["description"] = "\n".join([clean_text(p.text) for p in paragraphs])
            
            # Find all headers (h2, h3, h4, etc.) in the content div
            headers = content_div.find_all(['h2', 'h3', 'h4', 'h5', 'h6'])
            
            for header in headers:
                header_text = clean_text(header.text)
                description = []
                
                # Collect all paragraph siblings until the next header or end of content
                for sibling in header.find_next_siblings():
                    if sibling.name in ['h2', 'h3', 'h4', 'h5', 'h6']:
                        break
                    if sibling.name == 'p':
                        description.append(extract_text_from_element(sibling))
                
                parse_data["basic_info"].append({
                    "name": header_text,
                    "description": [item for item in description if item]
                })
                
    
    # extract tabular data
    tables = soup.find_all('table', class_=lambda x: x is not None and x.strip() != "")
    for table in tables:
        header = table.find_previous('h2')
        header_text = header.text.strip() if header else "No header found"

        # Extract headers
        headers = [th.text.strip() for th in table.find_all('th')]

        # Initialize the result list
        table_data = []

        # Track rowspans
        rowspan_tracker = [0] * len(headers)

        for row in table.find_all('tr')[1:]:  # Skip the header row
            cells = row.find_all(['td', 'th'])
            row_data = {}
            col_index = 0

            for cell in cells:
                while col_index < len(headers) and rowspan_tracker[col_index] > 0:
                    if table_data:
                        row_data[headers[col_index]] = table_data[-1].get(headers[col_index], "")
                    rowspan_tracker[col_index] -= 1
                    col_index += 1

                if col_index < len(headers):
                    rowspan = int(cell.get('rowspan', 1))
                    colspan = int(cell.get('colspan', 1))

                    for _ in range(colspan):
                        if col_index < len(headers):
                            row_data[headers[col_index]] = cell.text.strip()
                            if rowspan > 1:
                                rowspan_tracker[col_index] = rowspan - 1
                            col_index += 1

            if row_data:  # Avoid adding empty rows
                table_data.append(row_data)

        parse_data[header_text] = table_data


    
    # Initialize an empty list to store the FAQs
    faqs = []
    # Find all the question and answer pairs
    faq_div = soup.find('div', class_='cdcms_faqs')
    if faq_div:
        questions = faq_div.find_all('p', class_='accordio')
        answers = faq_div.find_all('div', class_='liv')

        # Iterate through the questions and answers and store them in a structured format
        for question, answer in zip(questions, answers):
            faq_item = {
                'question': question.get_text(strip=True).replace("Ques. ", ""),
                'answer': answer.get_text(strip=True).replace("Ans. ", "")
            }
            faqs.append(faq_item)
    parse_data['faqs']=faqs
    return parse_data

name = 'SRM'
url = 'https://collegedunia.com/university/25896-srm-institute-of-science-and-technology-srmist-chennai'

extracted = extract(name, url)
Scraper(extracted)

In [24]:
def search_college_on_duckduckgo(college_name):
    query = f"{college_name} site:collegedunia.com"
    
    # Create a DDGS object and perform a search
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=1)
    res=[]    
    if results:
        for result in results:
            res.append(result['href'])
        return res
    else:
        return "No CollegeDunia link found in the search results."

name = 'college_names6.csv'
df = pd.read_csv(name)
df['College Name'][0]

In [26]:
import pandas as pd
name = 'college_names.csv'
df = pd.read_csv(name)
df['College Name'][0]


for l in range(76, 88):
    name = df['College Name'][l]
    for web in search_college_on_duckduckgo(name):
        if web:
            extracted = extract(name, web)
        else:
            print(name)
        with open(f"{name}.json",'w',encoding='utf-8') as js:
            json.dump(scrape(extracted), js, ensure_ascii=False, indent=4)
            l
    

RatelimitException: https://duckduckgo.com/ 202 Ratelimit